### Carga y preparación de datos

In [ ]:
import polars as pl
import polars.selectors as cs

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import os

In [2]:
# carga de datos con polars

df_polars = pl.read_parquet("..//data//raw//sample_data_0006_part_00.parquet")

# Eliminacion de duplicados
df_polars = df_polars.unique()
df_polars.head(3)

merchant_id,_id,subsidiary,transaction_date,account_number,user_id,transaction_amount,transaction_type
str,str,str,datetime[ns],str,str,"decimal[24,8]",str
"""817d18cd3c31e40e9bff0566baae77…","""ffe2f9e58bd12be2519ec01a5316d5…","""02c54fef44bb992b3066e69309cac6…",2021-11-30 15:12:01,"""189bd9cc5a482ad92bc2914e964180…","""12162f290edcb3f1ea3099502af45f…",118.88910024,"""DEBITO"""
"""817d18cd3c31e40e9bff0566baae77…","""6664a8980112440bf1f8331bc1b82d…","""6535d7b0ce4d316b845359940ff9f5…",2021-09-04 10:19:53,"""ee24335b82d844439a70cc573767bb…","""f6c0b3857e1687571cb02d4e0dd548…",214.00038044,"""DEBITO"""
"""817d18cd3c31e40e9bff0566baae77…","""74dd0e180aa75af3493bac9ea1d54b…","""aae6146fd79e095a4ae4bf64d62e5a…",2021-08-16 12:10:15,"""84f5a778aee4e740ad3198f3422590…","""b11787c54390c86c2bbb47995a27b3…",118.88910024,"""DEBITO"""


In [3]:
# Ordenar por user y fecha
df_polars = df_polars.sort(["user_id", "transaction_date"]) 

df_polars.head()

merchant_id,_id,subsidiary,transaction_date,account_number,user_id,transaction_amount,transaction_type
str,str,str,datetime[ns],str,str,"decimal[24,8]",str
"""838a8fa992a4aa2fb5a0cf8b15b637…","""ecba8ff3dc6593175188301c8ae1ad…","""1c9c7c96fcd19d2063bb47e01c5df9…",2021-01-26 18:29:08,"""3fadf4b282c09463f16c6ebcc21c83…","""000002373d5835d0e53b7872242407…",29.72227506,"""CREDITO"""
"""838a8fa992a4aa2fb5a0cf8b15b637…","""55c4b353af003307c4ce801c9592c8…","""1c9c7c96fcd19d2063bb47e01c5df9…",2021-02-23 08:37:45,"""3fadf4b282c09463f16c6ebcc21c83…","""000002373d5835d0e53b7872242407…",71.33346014,"""CREDITO"""
"""838a8fa992a4aa2fb5a0cf8b15b637…","""f7a743f69702e6ce077074a43550ac…","""1c9c7c96fcd19d2063bb47e01c5df9…",2021-04-15 10:59:10,"""3fadf4b282c09463f16c6ebcc21c83…","""000002373d5835d0e53b7872242407…",23.77782004,"""CREDITO"""
"""075d178871d8d48502bf1f54887e52…","""d531b1fdca8cf53ad20525e6057632…","""24f755d1de59e3983ebfaf47f760cc…",2021-03-10 11:30:55,"""10e6890a86c790e6af17dc99f391a6…","""000004f4a6f3ac93f454a5dc04b2a2…",594.44550123,"""CREDITO"""
"""817d18cd3c31e40e9bff0566baae77…","""3c5e7b1b155417152b9ef87763eed1…","""80a2ee488271394873b4f5d993ca99…",2021-11-05 18:48:15,"""10e6890a86c790e6af17dc99f391a6…","""000004f4a6f3ac93f454a5dc04b2a2…",178.33365037,"""DEBITO"""


### Feature engineering

In [4]:
#crear columna date (día calendario)
df_polars = df_polars.with_columns(pl.col("transaction_date").dt.date().alias("date"))

agg_day = (
    df_polars.group_by(["user_id", "date"])
      .agg([
          pl.count("transaction_amount").alias("count_day"),
          pl.sum("transaction_amount").alias("sum_day"),
          pl.mean("transaction_amount").alias("mean_day"),
          pl.std("transaction_amount").alias("std_day"),
          pl.max("transaction_amount").alias("max_day")
      ])
)
agg_day.shape

(9214420, 7)

In [5]:
#  Número de transacciones por dia
agg_day.sort("count_day", descending=True).head(10)

user_id,date,count_day,sum_day,mean_day,std_day,max_day
str,date,u32,"decimal[24,8]",f64,f64,"decimal[24,8]"
"""71aa1651d9adc0b226107084fba3d7…",2021-02-07,288,6018.29227558,20.896848,44.253339,237.77820049
"""e2d6158cc2b6467a47223cb2629f03…",2021-02-02,201,1347.24890574,6.702731,7.65944,92.96889861
"""684202021037f203f2f2258f5f1819…",2021-01-20,200,1192.99624218,5.964981,0.252019,9.51112801
"""01c2e0350f7d77f4084502861b4989…",2021-02-05,200,1189.37488023,5.946874,0.000971,5.94802168
"""5e674596af22a66e826bf15b2a363c…",2021-02-10,200,2690.41515966,13.452076,10.43417,59.44455012
"""4677a8b53fc1ef1e16007968cdfcb2…",2021-02-12,200,1212.07437654,6.060372,1.156636,17.83336503
"""e77fc30bb3bec8bf45a35b535b2920…",2021-01-17,200,2634.17467079,13.170873,10.110278,59.44455012
"""f578e090182426cd7196f66c0f77e9…",2021-02-16,200,1218.61327706,6.093066,2.101682,35.66673007
"""f9a10e0f56a96d244feadda0194fcb…",2021-02-02,200,1188.89100200,5.944455,0.0,5.94445501


In [6]:
# Transacciones por dia en dinero
agg_day.sort("sum_day", descending=True).head(10)

user_id,date,count_day,sum_day,mean_day,std_day,max_day
str,date,u32,"decimal[24,8]",f64,f64,"decimal[24,8]"
"""cfa366b65fa843bf78ca52f9524e12…",2021-02-11,38,17559.19607174,462.084107,429.593394,1188.88981358
"""0290610111640c1d5b863240bb81d3…",2021-10-07,7,17238.91953584,2462.702791,870.179112,2972.22750618
"""22e351bd70fdb319bbc102f8de27b7…",2021-08-19,5,16050.02853335,3210.005707,0.0,3210.00570667
"""7cac676a8d21f4fb7a66d4966dd3a1…",2021-02-11,28,14887.89590055,531.710568,328.956469,1188.88981358
"""1d46b2330c1fdf739c0c3fcc511f35…",2021-02-11,33,14276.19245755,432.611893,259.020522,598.01217424
"""c671cb272294b914538f1f40b62edd…",2021-08-30,11,12007.79912494,1091.618102,218.045215,1188.89100247
"""7cac676a8d21f4fb7a66d4966dd3a1…",2021-02-09,35,11978.01383853,342.228967,295.480751,594.44550123
"""56dee2f91dc3be62894d127160fe25…",2021-11-17,5,11888.91002470,2377.782005,0.0,2377.78200494
"""8e09da7674c0e15b6cc367b21858bc…",2021-08-01,9,11888.91002469,1320.990003,649.673305,2377.78200494


### Calcular percentiles y aplicar regla heurística bajo mi percepción del negocio

Estamos buscando usuarios que en un mismo día:

* Hacen muchas transacciones pequeñas (fraccionamiento).
* El total diario de dinero es alto (la suma sigue siendo grande).
* Pero cada transacción individual tiene un monto bajo (menor al promedio general).

In [7]:
# 1. Calcular percentiles globales (sobre agg_day)
p95_count = float(agg_day.select(pl.col("count_day")).quantile(0.95)[0,0]) # Número alto en transacciones
p25_mean  = float(agg_day.select(pl.col("mean_day")).quantile(0.25)[0,0]) # Montos medios en transacciones por dia
p90_sum   = float(agg_day.select(pl.col("sum_day")).quantile(0.90)[0,0]) # Sumas altas de transacciones individuales por dia

p95_count, p25_mean, p90_sum

(2.0, 47.55564009, 594.44550123)

In [8]:
# 2. Aplicar reglas
agg_day = agg_day.with_columns([
    (pl.col("count_day") >= p95_count).alias("high_count"),
    (pl.col("mean_day") <= p25_mean).alias("low_mean"),
    (pl.col("sum_day") >= p90_sum).alias("high_sum")
])

agg_day = agg_day.with_columns(
    (pl.col("high_count") & pl.col("low_mean") & pl.col("high_sum")).alias("flag_heuristic")
)

# 3. Ver top sospechosos por conteo
heur_suspects = agg_day.filter(pl.col("flag_heuristic") == True).sort("count_day", descending=True)
heur_suspects.select(["user_id","date","count_day","sum_day","mean_day"]).head(10)


user_id,date,count_day,sum_day,mean_day
str,date,u32,"decimal[24,8]",f64
"""71aa1651d9adc0b226107084fba3d7…",2021-02-07,288,6018.29227558,20.896848
"""e2d6158cc2b6467a47223cb2629f03…",2021-02-02,201,1347.24890574,6.702731
"""684202021037f203f2f2258f5f1819…",2021-01-20,200,1192.99624218,5.964981
"""01c2e0350f7d77f4084502861b4989…",2021-02-05,200,1189.37488023,5.946874
"""5e674596af22a66e826bf15b2a363c…",2021-02-10,200,2690.41515966,13.452076
"""4677a8b53fc1ef1e16007968cdfcb2…",2021-02-12,200,1212.07437654,6.060372
"""e77fc30bb3bec8bf45a35b535b2920…",2021-01-17,200,2634.17467079,13.170873
"""f578e090182426cd7196f66c0f77e9…",2021-02-16,200,1218.61327706,6.093066
"""f9a10e0f56a96d244feadda0194fcb…",2021-02-02,200,1188.89100200,5.944455


In [9]:
agg_day

user_id,date,count_day,sum_day,mean_day,std_day,max_day,high_count,low_mean,high_sum,flag_heuristic
str,date,u32,"decimal[24,8]",f64,f64,"decimal[24,8]",bool,bool,bool,bool
"""3a56842f181ce3ec1ef4f0614fc20a…",2021-07-21,1,95.11128019,95.11128,null,95.11128019,false,false,false,false
"""ae0499ba2a6605abf0f7fe97f7aff2…",2021-10-28,1,23.77782004,23.77782,null,23.77782004,false,true,false,false
"""d8f61ce32af3bd7ce8831aa43a8ae7…",2021-01-10,3,17.83336503,5.944455,0.0,5.94445501,true,true,false,false
"""33f6a3f0d7d8a42d462e47a08b419b…",2021-05-18,1,166.44474034,166.44474,null,166.44474034,false,false,false,false
"""a74ff5c2bdbf1419dfb8a106e8f7c5…",2021-06-06,1,118.88910024,118.8891,null,118.88910024,false,false,false,false
…,…,…,…,…,…,…,…,…,…,…
"""ced4287fe65327e127f7f56bb1a01d…",2021-03-13,1,504.44645234,504.446452,null,504.44645234,false,false,false,false
"""1774e875cb1f4cd84079be7863c852…",2021-02-06,1,11.88891002,11.88891,null,11.88891002,false,true,false,false
"""0c40077d60db5120d51bcb3039eaa0…",2021-08-12,1,618.22332128,618.223321,null,618.22332128,false,false,true,false


Inspeccionar ejemplos: ver las transacciones concretas de usuarios sospechosos

In [10]:
# Elegir top 5 sospechosos para inspección
top_users = heur_suspects.select("user_id").unique().head(5).to_pandas()["user_id"].tolist()
top_users

['5e674596af22a66e826bf15b2a363cce',
 '6db2a61ce76075ef983a4826faa73f81',
 '328e7e1a0c72332e3011abf5d08a855a',
 '467376bcfd52781f39852c6acc988d28',
 'c47ee7710fa3e2212a917611bf70230c']

In [11]:
# Mostrar transacciones crudas de esos usuarios (ordenadas por fecha)
for u in top_users:
    print("=== User:", u, "===")
    user_tx = df_polars.filter(pl.col("user_id") == u).sort("transaction_date")
    display(user_tx.select(["transaction_date","transaction_amount","transaction_type","merchant_id"]).to_pandas().head(5))


=== User: 5e674596af22a66e826bf15b2a363cce ===


,transaction_date,transaction_amount,transaction_type,merchant_id
0,2021-01-26 16:07:12,5.94683279,CREDITO,838a8fa992a4aa2fb5a0cf8b15b63755
1,2021-01-26 16:07:22,5.94802168,CREDITO,838a8fa992a4aa2fb5a0cf8b15b63755
2,2021-01-26 16:10:05,5.94445501,CREDITO,838a8fa992a4aa2fb5a0cf8b15b63755
3,2021-01-26 16:10:08,5.94564390,CREDITO,838a8fa992a4aa2fb5a0cf8b15b63755
4,2021-01-26 16:10:23,5.94683279,CREDITO,838a8fa992a4aa2fb5a0cf8b15b63755


=== User: 6db2a61ce76075ef983a4826faa73f81 ===


,transaction_date,transaction_amount,transaction_type,merchant_id
0,2021-01-28 10:36:25,5.94683279,CREDITO,838a8fa992a4aa2fb5a0cf8b15b63755
1,2021-01-28 10:52:44,5.95039946,CREDITO,838a8fa992a4aa2fb5a0cf8b15b63755
2,2021-01-28 10:58:56,5.95039946,CREDITO,838a8fa992a4aa2fb5a0cf8b15b63755
3,2021-01-28 11:11:28,5.94445501,CREDITO,838a8fa992a4aa2fb5a0cf8b15b63755
4,2021-01-28 11:20:00,5.94683279,CREDITO,838a8fa992a4aa2fb5a0cf8b15b63755


=== User: 328e7e1a0c72332e3011abf5d08a855a ===


,transaction_date,transaction_amount,transaction_type,merchant_id
0,2021-01-31 13:37:24,5.94445501,DEBITO,838a8fa992a4aa2fb5a0cf8b15b63755
1,2021-01-31 14:00:40,5.94445501,DEBITO,838a8fa992a4aa2fb5a0cf8b15b63755
2,2021-01-31 15:14:12,5.94445501,DEBITO,838a8fa992a4aa2fb5a0cf8b15b63755
3,2021-01-31 15:57:12,5.94445501,DEBITO,838a8fa992a4aa2fb5a0cf8b15b63755
4,2021-01-31 16:04:27,5.94445501,DEBITO,838a8fa992a4aa2fb5a0cf8b15b63755


=== User: 467376bcfd52781f39852c6acc988d28 ===


,transaction_date,transaction_amount,transaction_type,merchant_id
0,2021-02-11 15:53:35,23.77782004,CREDITO,838a8fa992a4aa2fb5a0cf8b15b63755
1,2021-02-11 16:47:11,5.94445501,CREDITO,838a8fa992a4aa2fb5a0cf8b15b63755
2,2021-02-11 16:54:57,10.70001902,CREDITO,838a8fa992a4aa2fb5a0cf8b15b63755
3,2021-02-11 16:55:26,7.13334601,CREDITO,838a8fa992a4aa2fb5a0cf8b15b63755
4,2021-02-11 16:56:03,5.94445501,CREDITO,838a8fa992a4aa2fb5a0cf8b15b63755


=== User: c47ee7710fa3e2212a917611bf70230c ===


,transaction_date,transaction_amount,transaction_type,merchant_id
0,2021-01-09 13:12:40,89.16682518,DEBITO,075d178871d8d48502bf1f54887e52fe
1,2021-02-03 14:11:01,5.94445501,DEBITO,838a8fa992a4aa2fb5a0cf8b15b63755
2,2021-02-03 14:16:45,5.94445501,DEBITO,838a8fa992a4aa2fb5a0cf8b15b63755
3,2021-02-03 14:38:25,5.94445501,DEBITO,838a8fa992a4aa2fb5a0cf8b15b63755
4,2021-02-03 14:42:25,5.94445501,DEBITO,838a8fa992a4aa2fb5a0cf8b15b63755


### IsolationForest (no supervisado)

1. Preparar features (convertir agg_day a pandas)

In [3]:
agg_day = pl.read_parquet("..//data//processed//agg_day_with_flags.parquet")

In [4]:
features_pl = agg_day.select([
    "user_id","date","count_day","sum_day","mean_day","std_day","max_day"
])
features_pd = features_pl.to_pandas().fillna(0)
features_pd

,user_id,date,count_day,sum_day,mean_day,std_day,max_day
0,3a56842f181ce3ec1ef4f0614fc20acc,2021-07-21,1,95.11128019,95.111280,0.0,95.11128019
1,ae0499ba2a6605abf0f7fe97f7aff23c,2021-10-28,1,23.77782004,23.777820,0.0,23.77782004
2,d8f61ce32af3bd7ce8831aa43a8ae715,2021-01-10,3,17.83336503,5.944455,0.0,5.94445501
3,33f6a3f0d7d8a42d462e47a08b419bf1,2021-05-18,1,166.44474034,166.444740,0.0,166.44474034
4,a74ff5c2bdbf1419dfb8a106e8f7c519,2021-06-06,1,118.88910024,118.889100,0.0,118.88910024
...,...,...,...,...,...,...,...
9214415,ced4287fe65327e127f7f56bb1a01dae,2021-03-13,1,504.44645234,504.446452,0.0,504.44645234
9214416,1774e875cb1f4cd84079be7863c8522a,2021-02-06,1,11.88891002,11.888910,0.0,11.88891002
9214417,0c40077d60db5120d51bcb3039eaa0aa,2021-08-12,1,618.22332128,618.223321,0.0,618.22332128
9214418,350a0bdf5ec5ec7b96b0899dbe8a6531,2021-04-21,1,178.33365037,178.333650,0.0,178.33365037


2. Entrenar IsolationForest

In [5]:
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler

X = features_pd[["count_day","sum_day","mean_day","std_day","max_day"]].astype(float)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

model = IsolationForest(contamination=0.02, random_state=42, n_estimators=200)
model.fit(X_scaled)
features_pd["if_label"] = model.predict(X_scaled)        # -1 anomalía, 1 normal
features_pd["if_score"] = model.decision_function(X_scaled)  # mayor = más normal

* Obs: Se podria usar la base de datos sample_data_0007_part_00.parquet para predicción

In [7]:
# -1 = usuario anómalo o sospechoso
ml_suspects =  features_pd[features_pd["if_label"] == -1]

# Ordenar por puntaje de anomalía (menor score = más anómalo)
ml_suspects = ml_suspects.sort_values("if_score", ascending=True)

# Mostrar los top sospechosos
ml_suspects.head(10)[[
    "user_id", "date",
    "count_day", "sum_day", "mean_day",
    "std_day", "max_day", "if_score"
]]

,user_id,date,count_day,sum_day,mean_day,std_day,max_day,if_score
3363004,0290610111640c1d5b863240bb81d3b4,2021-10-07,7,17238.91953584,2462.702791,870.179112,2972.22750618,-0.152709
7315978,1c9cd25a45ea8edbe6b75094d7b2f458,2021-07-01,7,11651.13182420,1664.447403,989.949591,3091.11660642,-0.151574
7422437,8ded74c4bdac01be91dcd56e379d19d4,2021-10-12,6,9035.57161876,1505.928603,1227.881347,3091.11660642,-0.149875
8470248,a57bd17de30103e8415ea4cee68d0733,2021-10-26,9,11056.68632296,1228.520703,1129.446452,3210.00570667,-0.149309
2222084,dc3b0d7f51827bf098f15a3d53b64b93,2021-09-13,4,8678.90431803,2169.726080,1225.480795,3210.00570667,-0.148461
3119567,a68e6320a5082d0346139df6f65a348e,2021-11-25,5,9748.90622025,1949.781244,833.708491,3210.00570667,-0.148461
6235582,8c0f20e0e7c73940b7fe0b9b97dbd3cf,2021-07-01,4,8084.45881679,2021.114704,1250.692239,3091.11660642,-0.148179
1830447,415fa21168d1b303d147f3e6fb8b5e45,2021-10-19,6,8560.01521778,1426.669203,1105.093213,2853.33840593,-0.147897
3017507,1c61c27340a8b59f350c56f8f8dbf980,2021-03-28,4,8952.34924861,2238.087312,1108.004530,2972.22750618,-0.147614
606838,c5923c6836b1b6061202f5ccf42a3f50,2021-10-02,4,9630.01712000,2407.504280,938.019179,3210.00570667,-0.147614


### Guardar resultados

In [ ]:
# Guardar outputs (parquet) 7para dashboard


agg_day.write_parquet("..//data//processed//agg_day_with_flags.parquet")
heur_suspects.write_parquet("..//data//processed//heur_suspects_results.parquet")
features_pd.to_csv("..//data//processed//isolation_forest_results.csv")

# guardar artefactos
joblib.dump(model, os.path.join("..//models", "isolation_forest.joblib"))
joblib.dump(scaler, os.path.join("..//models", "scaler.joblib"))